In [56]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from keras.layers import Dropout

In [22]:
from sklearn.metrics import classification_report 
from sklearn.metrics import f1_score ,accuracy_score

In [49]:
import cv2, imutils
def extract_color_histogram(image, bins=(8, 8, 8)):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, bins,
    [0, 180, 0, 256, 0, 256])
    if imutils.is_cv2():
        hist = cv2.normalize(hist)
    else:
        cv2.normalize(hist, hist)
    return hist.flatten()

In [24]:
import os
import pandas as pd
cwd = os.getcwd()
print(cwd)
df = pd.read_csv(cwd + "\\..\\data_csv\\all_data.csv")
print(df)

c:\Users\Pongsakorn\OneDrive\Desktop\Working\FacialLandMarks\facial_cleandata\neural_network
                                 image   emotion
0      facial-expressions_2868588k.jpg     anger
1      facial-expressions_2868585k.jpg  surprise
2      facial-expressions_2868584k.jpg   disgust
3      facial-expressions_2868582k.jpg      fear
4               Aaron_Eckhart_0001.jpg   neutral
...                                ...       ...
34670      SURPRISE/surprise (988).jpg  SURPRISE
34671       SURPRISE/surprise (99).jpg  SURPRISE
34672      SURPRISE/surprise (993).jpg  SURPRISE
34673      SURPRISE/surprise (994).png  SURPRISE
34674      SURPRISE/surprise (996).jpg  SURPRISE

[34675 rows x 2 columns]


In [50]:
raw_images =  []
labels = []
for i, row in df.iterrows():
    image_path = row['image']
    label = row['emotion']
    full_image_path = os.path.dirname(cwd) + "\\cleaned_images\\" + image_path
    image = cv2.imread(full_image_path)
    pixels = extract_color_histogram(image)

    raw_images.append(pixels)
    labels.append(label)
    if i > 0 and i % 1000 == 0: print('[INFO] processed {}/{}'.format(i, len(df)))

[INFO] processed 1000/34675
[INFO] processed 2000/34675
[INFO] processed 3000/34675
[INFO] processed 4000/34675
[INFO] processed 5000/34675
[INFO] processed 6000/34675
[INFO] processed 7000/34675
[INFO] processed 8000/34675
[INFO] processed 9000/34675
[INFO] processed 10000/34675
[INFO] processed 11000/34675
[INFO] processed 12000/34675
[INFO] processed 13000/34675
[INFO] processed 14000/34675
[INFO] processed 15000/34675
[INFO] processed 16000/34675
[INFO] processed 17000/34675
[INFO] processed 18000/34675
[INFO] processed 19000/34675
[INFO] processed 20000/34675
[INFO] processed 21000/34675
[INFO] processed 22000/34675
[INFO] processed 23000/34675
[INFO] processed 24000/34675
[INFO] processed 25000/34675
[INFO] processed 26000/34675
[INFO] processed 27000/34675
[INFO] processed 28000/34675
[INFO] processed 29000/34675
[INFO] processed 30000/34675
[INFO] processed 31000/34675
[INFO] processed 32000/34675
[INFO] processed 33000/34675
[INFO] processed 34000/34675


In [51]:
from sklearn.model_selection import train_test_split
import numpy as np
X_train, X_test, y_train, y_test = train_test_split(np.array(raw_images), np.array(labels), random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)


In [58]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)
y_test = le.transform(y_test)
y_val = le.transform(y_val)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)

In [63]:
import tensorflow as tf
def classification_model():
    model = Sequential()
    model.add(Dense(124, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dropout(0.2))
    model.add(Dense(124, activation='relu'))
    model.add(Dropout(0.2)) 
    model.add(Dense(32, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='softmax'))
    opt = tf.keras.optimizers.Adam(lr=0.0001, decay=1e-6)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [66]:
model = classification_model()
model.fit(X_train, y_train, validation_data=(X_val, y_val,), epochs=10, verbose=2)

Epoch 1/10
610/610 - 2s - loss: 2.5263 - accuracy: 0.1371 - val_loss: 2.3122 - val_accuracy: 0.1913 - 2s/epoch - 3ms/step
Epoch 2/10
610/610 - 1s - loss: 2.2839 - accuracy: 0.1679 - val_loss: 2.2447 - val_accuracy: 0.2019 - 1s/epoch - 2ms/step
Epoch 3/10
610/610 - 1s - loss: 2.2485 - accuracy: 0.1803 - val_loss: 2.2254 - val_accuracy: 0.1959 - 1s/epoch - 2ms/step
Epoch 4/10
610/610 - 1s - loss: 2.2329 - accuracy: 0.1886 - val_loss: 2.2172 - val_accuracy: 0.2053 - 1s/epoch - 2ms/step
Epoch 5/10
610/610 - 1s - loss: 2.2272 - accuracy: 0.1885 - val_loss: 2.2123 - val_accuracy: 0.2032 - 1s/epoch - 2ms/step
Epoch 6/10
610/610 - 1s - loss: 2.2210 - accuracy: 0.1943 - val_loss: 2.2094 - val_accuracy: 0.2039 - 1s/epoch - 2ms/step
Epoch 7/10
610/610 - 1s - loss: 2.2165 - accuracy: 0.1944 - val_loss: 2.2075 - val_accuracy: 0.2062 - 1s/epoch - 2ms/step
Epoch 8/10
610/610 - 1s - loss: 2.2129 - accuracy: 0.1934 - val_loss: 2.2053 - val_accuracy: 0.2070 - 1s/epoch - 2ms/step
Epoch 9/10
610/610 - 1s 

In [67]:
value = model.predict(X_test)
y_pred =np.argmax(value,axis=1)
y_true = np.argmax(y_test,axis=1)

In [68]:
print(classification_report(y_true,y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       579
           1       0.00      0.00      0.00       536
           2       0.00      0.00      0.00       636
           3       0.23      0.07      0.11       641
           4       0.15      0.15      0.15      1120
           5       0.18      0.37      0.24      1164
           6       0.16      0.01      0.02       741
           7       0.19      0.20      0.20       803
           8       0.00      0.00      0.00        46
           9       0.00      0.00      0.00         3
          10       0.00      0.00      0.00         3
          11       0.00      0.00      0.00         3
          12       0.00      0.00      0.00      1066
          13       0.23      0.74      0.35      1249
          14       0.00      0.00      0.00        17
          15       0.00      0.00      0.00        62

    accuracy                           0.20      8669
   macro avg       0.07   

C:\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
